# Data Indexing and Selection

1. **Selección de datos en SERIE**
2. **Selección de datos en DataFrame**
   

Hemos visto en detalle los métodos y herramientas para acceder, establecer y modificar valores en matrices NumPy.
Estos incluyen indexación (por ejemplo, ``arr[2, 1]``), corte (por ejemplo, ``arr[:, 1:5]``), enmascaramiento (por ejemplo, ``arr[arr > 0]``), indexación de fantasía (por ejemplo, ``arr[0, [1, 5]]``), y combinaciones de los mismos (por ejemplo, ``arr[:, [1, 5]]``).
**Aquí veremos formas similares de acceder y modificar valores en objetos Pandas ``Series`` y ``DataFrame``.
Si has usado los patrones de NumPy, los patrones correspondientes en Pandas te resultarán muy familiares, aunque hay algunas peculiaridades que debes tener en cuenta.

Empezaremos con el caso simple del objeto unidimensional ``Series``, y luego pasaremos al más complicado objeto bidimensional ``DataFrame``.

## **1. Selección de datos en SERIE**

Como vimos en la sección anterior, un objeto **``Series`` actúa en muchos aspectos como un array unidimensional de NumPy, y en muchos aspectos como un diccionario estándar de Python.**
Si mantenemos en mente estas dos analogías superpuestas, nos ayudará a entender los patrones de indexación y selección de datos en estos arrays.

### Series como diccionario

Al igual que un diccionario, el objeto ``Series`` proporciona una correspondencia entre una colección de claves y una colección de valores:

In [2]:
import pandas as pd
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [2]:
data['b']

0.5

También podemos utilizar expresiones y métodos de Python tipo diccionario para examinar las claves/índices y los valores:

In [3]:
'b' in data # Sería: 'b' está en data? True = porque sí está

True

In [4]:
data.keys() # Devuelve los índices estilo diccionario

Index(['a', 'b', 'c', 'd'], dtype='object')

In [5]:
data.index # Este es igual que keys() de Python, pero el .index es propio de PANDAS

Index(['a', 'b', 'c', 'd'], dtype='object')

In [6]:
list(data.items()) # Muestra el VALOR y el INDICE

[('a', 0.25), ('b', 0.5), ('c', 0.75), ('d', 1.0)]

> AGREGAR NUEVO ÍNDICE y VALOR:

Los objetos ``Series`` pueden incluso modificarse con una sintaxis similar a la de un diccionario.
Del mismo modo que se puede ampliar un diccionario asignando una nueva clave, se puede ampliar una ``Serie`` asignando un nuevo valor de índice:

In [22]:
data['e'] = 1.25
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

> MODIFICAR VALORES SEGÚN ÍNDICE:

In [3]:
data['d'] = 1.80

In [4]:
data

a    0.25
b    0.50
c    0.75
d    1.80
dtype: float64

Esta fácil mutabilidad de los objetos es una característica conveniente: bajo el capó, Pandas está tomando decisiones sobre la disposición de la memoria y la copia de datos que pueda ser necesario realizar; el usuario generalmente no necesita preocuparse por estos temas.

### Series como array unidimensional

> **SI TENEMOS CONJUNTO DE DATOS PEQUEÑOS, PUEDEN UTILIZARSE ESTOS TIPOS DE INDEXACIÓN, PERO SI ES UN CONJUNTO DE DATOS GRANDES, PARA EVITAR ERRORES, DEBE USARSE LOC - ILOC - IX**

Una ``Serie`` se basa en esta interfaz tipo diccionario y proporciona una selección de elementos tipo array a través de los mismos mecanismos básicos que los arrays de NumPy, es decir, *rebanadas*, *enmascaramiento* e *índice de fantasía*.
Algunos ejemplos son los siguientes:

> CORTE POR ÍNDICE EXPLÍCITO:

In [18]:
# corte por índice explícito
data['a':'c'] # Esto muestra todos los keys incluido "a" y "c"

a    0.25
b    0.50
c    0.75
dtype: float64

> CORTE POR ÍNDICE IMPLÍCITO:

In [19]:
# corte por índice entero implícito
data[0:2] # la diferencia con el anterior es que éste por posición no incluye el último 

a    0.25
b    0.50
dtype: float64

In [20]:
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

> ENMASCARAMIENTO - SELECCIÓN CONDICIONAL

In [21]:
# enmascaramiento - SELECCIÓN CONDICIONAL
data[(data > 0.3) & (data < 0.8)]

b    0.50
c    0.75
dtype: float64

> SELECCIONAR COLUMNAS QUE QUIERO VER DE LA SERIE [[]]

- `data[['a', 'e']]` >> Selecciono columnas que quiero ver, únicamente me muestra las nombradas 'a' y 'e'

- `data['a':'c']` >> Es diferente porque acá se realiza un corte por índice explícito, y muestra todos los valores incluidos DESDE la 'a' A la 'c'. 

**IMP !!** > Si necesito esas columnas seleccionadas no más, puedo guardarlas en una variable >>  `colum_esp = data[['a', 'e']]` y luego operar solo con esos datos

In [23]:
# indexación sofisticada
data[['a', 'e']] # Este tiene DOBLE CORCHETES y es para SELECCIONAR COLUMNAS que quiero ver

a    0.25
e    1.25
dtype: float64

Entre ellos, el troceado puede ser la fuente de mayor confusión.

**Tenga en cuenta que cuando se corta con un índice explícito (es decir, ``data['a':'c']``), el índice final está *incluido* en la rebanada, mientras que cuando se corta con un índice implícito (es decir, ``data[0:2]``), el índice final está *excluido* de la rebanada.**

### **Indexadores: loc, iloc, e ix**

> **Para GRANDES ATRIBUTOS de datos se usan estos ATRIBUTOS y no las funciones anteriores que vimos**

Estas convenciones de corte e indexación pueden ser fuente de confusión.
Por ejemplo, si tu ``Serie`` tiene un índice entero explícito, una operación de indexación como **`datos[1]` usará los índices explícitos, mientras que una operación de corte como `datos[1:3]` usará el índice implícito al estilo Python.**

In [28]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

1    a
3    b
5    c
dtype: object

- 
  ÍNDICE EXPLÍCITO

In [29]:
# explicit index when indexing
data[1] #Acá llama al valor del índice que le pase yo, por eso devuelve "a" y no "b" que sería el índice 1 natural

'a'

- ÍNDICE IMPLÍCITO

In [30]:
# implicit index when slicing
data[1:6] # Acá en cambio devuelve la posición natural según índice ímplicito, y NO según el índice que le pase yo

# Si último valor no existe, por ej: 6, muestra los valores desde la posición que le digo hasta el infinito

3    b
5    c
dtype: object

Debido a esta confusión potencial en el caso de índices enteros, Pandas proporciona algunos atributos *indexer* especiales que exponen explícitamente ciertos esquemas de indexación.
No se trata de métodos funcionales, sino de atributos que exponen una interfaz de corte particular para los datos de la ``Serie``.

> #### **LOC = INDICE EXPLÍCITO QUE YO CREO**

En primer lugar, el atributo **`loc`` permite la indexación y el corte que siempre hace referencia al índice explícito, es decir, al índice que yo cree:**

In [31]:
data

1    a
3    b
5    c
dtype: object

In [32]:
data.loc[1]

'a'

In [33]:
data.loc[1:3]

1    a
3    b
dtype: object

> #### **ILOC = INDICE IMPLICITO NATURAL DE PYTHON**
> 
El atributo ``iloc`` permite indexar y rebanar que siempre hace referencia al índice implícito estilo Python:

In [34]:
data

1    a
3    b
5    c
dtype: object

In [35]:
data.iloc[1]

'b'

In [36]:
data.iloc[1:3] # Asi le pongo 30 como fin, NO da error, y me devuelve hasta el último elemento que hay en la serie

3    b
5    c
dtype: object

Un tercer atributo de indexación, ``ix``, es un híbrido de los dos, y para los objetos ``Series`` es equivalente a la indexación estándar basada en ``[]``.
El propósito del indexador ``ix`` se hará más evidente en el contexto de los objetos ``DataFrame``, que discutiremos en un momento.

Un principio rector del código Python es que "lo explícito es mejor que lo implícito".
La naturaleza explícita de ``loc`` y ``iloc`` los hace muy útiles para mantener un código limpio y legible; especialmente en el caso de índices de enteros, **recomiendo usarlos tanto para hacer el código más fácil de leer y entender, como para prevenir bugs sutiles debidos a la convención mixta de indexación/rebanado.**

## **2. Selección de datos en DataFrame**
Recordemos que un ``DataFrame`` actúa en muchos aspectos como una matriz bidimensional o estructurada, y en otros como un diccionario de estructuras ``Series`` que comparten el mismo índice.
Estas analogías pueden ser útiles para tener en cuenta a medida que exploramos la selección de datos dentro de esta estructura.

### DataFrame como diccionario

La primera analogía que vamos a considerar es el ``DataFrame`` como diccionario de objetos ``Series`` relacionados.
Volvamos a nuestro ejemplo de las áreas y poblaciones de los estados:

In [71]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})
# cities = pd.Series({'California': "Sacramento", 'Texas': "Otra ciudad 1",
#                  'New York': "Albany", 'Florida': "Miami",
#                  'Illinois': "Otra ciudad 2"})
data = pd.DataFrame({'area':area, 'population':population})
data

,area,population
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


In [ ]:
# data.reset_index()

#### **Se puede acceder a las ``Series`` individuales que componen las columnas del ``DataFrame`` mediante:**
1. `data['population']` >> Indexación de tipo diccionario del nombre de la columna
2. `data.population` >> Atributo para acceder según el nombre de las columnas

> ##### 1. Indexación de tipo diccionario del nombre de la columna:

**IMP !! >** SE RECOMIENDA UTILIZAR ESTE MÉTODO PARA ACCEDER POR LAS ETIQUETAS DE LAS COLUMNAS, YA QUE SI EL NOMBRE ES SIMILAR A UNA PALABRA RESERVADA... Poner el DataFrame + punto + nombre de la columna, podría dar error!! 

In [72]:
data['population'] # MÉTODO RECOMENDADO

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
Name: population, dtype: int64

> ##### 2. Atributo para acceder según el nombre de las columnas:

De forma equivalente, podemos utilizar el acceso de tipo atributo con nombres de columna que sean cadenas:

In [56]:
data.population # NO RECOMENDADO

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
Name: population, dtype: int64

>> ##### **.get()** >> Puede usarse en los dos casos para acceder a los datos de las columnas según índice

In [52]:
data.population.get('California', 'no existe')

38332521

In [51]:
data['population'].get('California', 'no existe')

38332521

> IMP!! >> OJO CON USAR PALABRAS RESERVADAS!! --- EJEMPLO: ***pop***

In [40]:
data.pop #OJO CON USAR PALABRAS RESERVADAS!! POP NO PODRÍA USARSE PORQUE PUEDE DAR ERROR

# LO CAMBIÉ A 'POPULATION' PERO ANTES LA SERIE Y LA COLUMNA SE LLAMABA 'POP'

<bound method DataFrame.pop of               area  population
California  423967    38332521
Texas       695662    26448193
New York    141297    19651127
Florida     170312    19552860
Illinois    149995    12882135>

Este acceso de columna de tipo atributo accede en realidad exactamente al mismo objeto que el acceso de tipo diccionario:

In [37]:
data.area is data['area'] # SIEMPRE USAR FORMATO data["area"] con corchetes !!

True

SINTAXIS = `data["nombre de columna"]`

Porque si tengo por ejemplo una columna que se llama suma. Y luego al llamarlo debo poner data.suma, me lo puede interpretar como la función sum(). 
Por eso, por las dudas siempre es mejor usar la forma entre corchetes para poder poner caracteres o espacios por ejemplo, que data. ... no te permite poner espacios por ej, si el nombre de la columna lo tiene, y si lo llamo sin espacio, me da error porque no sería lo mismo

Aunque se trata de una abreviatura útil, hay que tener en cuenta que no funciona en todos los casos.
Por ejemplo, **si los nombres de las columnas no son cadenas, o si los nombres de las columnas entran en conflicto con métodos del ``DataFrame``, este acceso tipo atributo no es posible.**
Por ejemplo, el ``DataFrame`` tiene un método ``pop()``, por lo que ``data.pop`` apuntará a éste en lugar de a la columna ``"pop"``:

In [38]:
data.pop is data['pop']

False

En particular, debe evitar la tentación de intentar asignar columnas mediante atributos (es decir, utilice ``datos['pop'] = z`` en lugar de ``datos.pop = z``).

Al igual que con los objetos ``Series`` comentados anteriormente, esta sintaxis de tipo diccionario también se puede utilizar para modificar el objeto, en este caso añadiendo una nueva columna:

In [57]:
data

,area,population
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


> #### **CREAR COLUMNA NUEVA CON RESULTADOS DE OPERACIONES DE OTRAS COLUMNAS**

Se puede crear una columna nueva con el resultado de dividir, multiplicar, sumar, etc, otras columnas del dataFrame, y te agrega el nombre de la columna con los resultados de cada fila

In [73]:
data['population'] / data['area']

California     90.413926
Texas          38.018740
New York      139.076746
Florida       114.806121
Illinois       85.883763
dtype: float64

Si quiero saber la densidad, puedo dividir los datos a través de los nombres de las columnas. 
Y si dada la operación anterior entre la serie 'population' y la serie 'area' me da una nueva serie, puedo asignar esta nueva a una nueva columna, de la siguiente forma directamente:

In [74]:
data['density'] = data['population'] / data['area'] # LO HAGO DIRECTAMENTE DE ESTA FORMA
data

,area,population,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


Esto muestra una vista previa de la sintaxis directa de la aritmética elemento a elemento entre objetos ``Series``; profundizaremos más en esto en [Operando con datos en Pandas](3_Operaciones-en-Pandas.ipynb).

### **DataFrame como matriz bidimensional**

Como se mencionó anteriormente, también podemos ver el ``DataFrame`` como una matriz bidimensional mejorada.
Podemos examinar la matriz de datos subyacente utilizando el atributo ``values``:

In [62]:
data.values

array([[4.23967000e+05, 3.83325210e+07, 9.04139261e+01],
       [6.95662000e+05, 2.64481930e+07, 3.80187404e+01],
       [1.41297000e+05, 1.96511270e+07, 1.39076746e+02],
       [1.70312000e+05, 1.95528600e+07, 1.14806121e+02],
       [1.49995000e+05, 1.28821350e+07, 8.58837628e+01]])

Con esta imagen en mente, se pueden hacer muchas observaciones familiares sobre el propio ``DataFrame``.
**Por ejemplo, podemos transponer el ``DataFrame`` completo para intercambiar filas y columnas:**.

> **.T =** Convierte las filas en columnas y las columnas en filas



In [75]:
data.T # convierte las filas en columnas y las columnas en filas

,California,Texas,New York,Florida,Illinois
area,4.239670e+05,6.956620e+05,1.412970e+05,1.703120e+05,1.499950e+05
population,3.833252e+07,2.644819e+07,1.965113e+07,1.955286e+07,1.288214e+07
density,9.041393e+01,3.801874e+01,1.390767e+02,1.148061e+02,8.588376e+01


Cuando se trata de la indexación de objetos ``DataFrame``, sin embargo, está claro que el estilo de indexación de diccionario de las columnas impide nuestra **capacidad de simplemente tratarlo como una matriz NumPy.**
En particular, al pasar un único índice a un array se accede a una fila:

In [76]:
data

,area,population,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [43]:
data.values[0] # Devuelve toda la primer fila 

array([4.23967000e+05, 3.83325210e+07, 9.04139261e+01])

Y pasando un índice a ``DataFrame`` devuelve la columna:

In [77]:
data['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

Por lo tanto, para la indexación estilo array, necesitamos otra convención.
Aquí Pandas utiliza de nuevo los indexadores ``loc``, ``iloc``, y ``ix`` mencionados anteriormente.
Usando el indexador ``iloc``, podemos indexar el array subyacente como si fuera un simple array NumPy (usando el índice implícito estilo Python), pero el índice ``DataFrame`` y las etiquetas de las columnas se mantienen en el resultado:

In [78]:
data

,area,population,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


> ##### **.iloc[ ]** = **Para buscar los datos por los índices implícitos de FILAS y COLUMNAS**

- 1ero: Se pone índice de las FILAS 
- 2do: Se pone el índices de las COLUMNAS

In [80]:
data.iloc[:4, 1:3] # NO da error si le pasamos un índice mayor al que tiene el DataFrame, sino, que te devuelve hasta el último índice

,population,density
California,38332521,90.413926
Texas,26448193,38.018740
New York,19651127,139.076746
Florida,19552860,114.806121


> ##### **.loc[ ]** = **Para buscar datos por los nombres/índices de las FILAS y de las COLUMNAS** 

Del mismo modo, utilizando el indexador ``loc`` podemos indexar los datos subyacentes en un estilo similar a un array pero utilizando el índice explícito y los nombres de las columnas:

In [96]:
data

,area,population,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [83]:
data.loc['Texas':'New York', 'population':'density'] # Incluye todo lo que le paso

,population,density
Texas,26448193,38.018740
New York,19651127,139.076746


Recordar que con los **DOS CORCHETES [[ ]]** puedo buscar columnas en concreto según su nombre

In [86]:
data.loc[['Texas', 'Florida']]

,area,population,density
Texas,695662,26448193,38.018740
Florida,170312,19552860,114.806121


>>> #### **INDEXACIÓN = MÁSCARA BOOLEANA con CONDICIONAL E INDEX POR COLUMNA**

Dentro de estos indexadores se puede utilizar cualquiera de los patrones de acceso a datos conocidos del estilo NumPy.
Por ejemplo, en el indexador ``loc`` podemos combinar enmascaramiento e indexación de fantasía como en lo siguiente:

In [92]:
data.loc[data.density > 100] # Opción NO recomendada. La correcta es = data.loc[data['density'] > 100] 

,area,population,density
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121


> **MÁSCARA BOOLEANA = Para ver las filas que cumplen con la condición por True or False** -- OPCIÓN 1

In [111]:
mask = data['density'] > 100 # Esto es para ver los True o False. Se puede hacer directamente con la opción 2
mask

California    False
Texas         False
New York       True
Florida        True
Illinois      False
Name: density, dtype: bool

In [114]:
data[mask][['population', 'density']] # Puedo elegir las columnas que quiero ver también

,population,density
New York,19651127,139.076746
Florida,19552860,114.806121


> **MÁSCARA BOOLEANA con LOC = Directamente para ver las filas que cumplen con la condición por True or False** -- OPCIÓN 2

In [93]:
data.loc[data['density'] > 100] 

,area,population,density
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121


- Puedo elegir las **COLUMNAS** en las cuales quiero ver esos datos > **OPCIÓN RECOMENDADA = 1 LÍNEA DE CÓDIGO**

In [94]:
data.loc[data['density'] > 100, ['population','density']] # Si solo quiero ver dichos datos pero de esas dos columnas

,population,density
New York,19651127,139.076746
Florida,19552860,114.806121


> #### **INDEXACIÓN PARA MODIFICAR VALORES:**

Cualquiera de estas convenciones de indexación también se puede utilizar para establecer o modificar valores; esto se hace de la manera estándar a la que puede estar acostumbrado de trabajar con NumPy:

>>> **.iloc( )**:


In [100]:
data.iloc[0, 2] = 90 #Cambiar el dato de esa posición, o sea: fila 0, columna 2 = density > 'California'
data

,area,population,density
California,420000,38332521,90.000000
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


>>> **.loc( )**:

SIEMPRE el nombre de la fila (índice) va primero y luego el nombre de las columnas (índice) == .iloc() pero con los índices implícitos

In [102]:
data.loc['California','area'] = 420000 # Cambiar el dato por nombre de fila y de columna

In [101]:
data

,area,population,density
California,420000,38332521,90.000000
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


Para mejorar tu fluidez en la manipulación de datos en Pandas, te sugiero que pases algún tiempo con un simple ``DataFrame`` y explores los tipos de indexación, troceado, enmascaramiento e indexación de fantasía que permiten estos diversos enfoques de indexación.

> ### **Convenciones de indexación adicionales**

Hay un par de convenciones de indexación adicionales que pueden parecer contradictorias con la discusión anterior, pero que sin embargo pueden ser muy útiles en la práctica.
**En primer lugar, mientras que *indexar* se refiere a columnas, *rebanar* se refiere a filas:**.

>>> **Búsqueda por FILAS = Devuelve todas las columnas pero de las filas especificadas**

In [118]:
data['Florida':'Illinois'] # Acá busca los índices que quiero y devuelve todas las columnas
                           # de esas FILAS especificadas

,area,population,density
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


>>> **Búsqueda con .loc() por FILAS y COLUMNAS = Especificar rango de filas y de columnas** 
OPCIÓN RECOMENDADA/MEJOR RENDIMIENTO CON .loc() en conjunto de datos muy grandes

* Si quiero filas o columnas enteras colocar **: (2 puntos)**
* Separar los rangos de filas y de columnas por la **, (coma)**

In [104]:
# EJEMPLO 1
data.loc['Texas': 'New York', 'area':'population'] # Acá devuelve todas las columnas
                              # hay que poner los : al principio SI o SI, si no da error

,area,population
Texas,695662,26448193
New York,141297,19651127


In [106]:
# EJEMPLO 2 - Con :
data.loc[:,'area':'population'] # Todas las filas por el : y las columnas SIN density porque no la incluí

,area,population
California,420000,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


Estas rebanadas también pueden referirse a filas por número en lugar de por índice:

In [119]:
data[1:3]

,area,population,density
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746


Del mismo modo, las operaciones de enmascaramiento directo también se interpretan por filas en lugar de por columnas:

In [120]:
data[data.density > 100]

,area,population,density
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121


Estas dos convenciones son sintácticamente similares a las de un array de NumPy, y aunque no se ajusten exactamente al molde de las convenciones de Pandas, son bastante útiles en la práctica.